In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakesoeare", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

1130496/1115394 [==============================] - 0s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# encode each character as an integer
# tockenizer converts text to lower case as default
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # char_level=True so that we get character level encoding rather than the default word-level encoding
tokenizer.fit_on_texts(shakespeare_text)

In [ ]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [ ]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [ ]:
# number of distinct characters
max_id = len(tokenizer.word_index)
# total number of characters
dataset_size = tokenizer.document_count

In [ ]:
max_id

39

In [ ]:
dataset_size

1115394

In [ ]:
# encode the full text so each character is represented by its ID ( subtract 1 to get IDs 0 to 38, rather than 1 to 39)
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

# splt data 90% vor training and 10 % for validation and test
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
# Chopping the Sequential Dataset into Multiple Windows
# we use the window() method to convert the 1 mil long sequence of characters into multiple equal windows of text 
# also named truncated backpropagation through time

n_steps = 100
window_length = n_steps + 1

dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [ ]:
# convert the nested dataset into a flat dataset using flat_map(). nested => {{1,2},{3,4,5,6}}, flat => {[1,2],[3,4],[5,6]}
# we call window_length on each window since all windows have exactly that length
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
# encode each character using one-hot vector
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
# add prefetching
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation = "softmax"))

])


In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset,epochs=20)

Epoch 1/20
31368/31368 [==============================] - 370s 12ms/step - loss: 1.6227
Epoch 2/20
31368/31368 [==============================] - 364s 12ms/step - loss: 1.5357
Epoch 3/20
31368/31368 [==============================] - 363s 12ms/step - loss: 1.5127
Epoch 4/20
31368/31368 [==============================] - 362s 12ms/step - loss: 1.5005
Epoch 5/20
31368/31368 [==============================] - 363s 12ms/step - loss: 1.4934
Epoch 6/20
31368/31368 [==============================] - 360s 11ms/step - loss: 1.4880
Epoch 7/20
31368/31368 [==============================] - 359s 11ms/step - loss: 1.4844
Epoch 8/20
31368/31368 [==============================] - 354s 11ms/step - loss: 1.4813
Epoch 9/20
31368/31368 [==============================] - 363s 12ms/step - loss: 1.4793
Epoch 10/20
31368/31368 [==============================] - 347s 11ms/step - loss: 1.4768
Epoch 11/20
31368/31368 [==============================] - 345s 11ms/step - loss: 1.4751
Epoch 12/20
31368/31368 [=====

In [ ]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts)) -1
  return tf.one_hot(X, max_id)

In [ ]:
#X_new = preprocess(["how are you"])
#Y_pred = model.predict_classes(X_new)
#y_pred = np.argmax(model.predict(X_new), axis=-1)
#tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

In [ ]:
def preprocess(texts):
    X=np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X,max_id)
 
def next_char(text,temperature=1):
    X_new=preprocess([text])
    y_proba=model.predict(X_new)[0,-1:,:]
    rescaled_logits=tf.math.log(y_proba)/temperature
    char_id=tf.random.categorical(rescaled_logits,num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
 
def complete_text(text,n_chars=50,temperature=1):
    for _ in range(n_chars):
        text+=next_char(text,temperature)
    return text

In [1]:
print("Some predicted texts for letter 'D' are as follows:\n ")
for i in range(3):
  print(complete_text('c'))
  print()

Some predicted texts for letter 'D' are as follows:
 


NameError: ignored